## - - 이전 코드 시작 - -

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 800 kB/s 
     |████████████████████████████████| 806 kB 7.9 MB/s 
     |████████████████████████████████| 2.8 MB 45.1 MB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
     |████████████████████████████████| 829 kB 52.8 MB/s 
     |████████████████████████████████| 398 kB 64.4 MB/s 
     |████████████████████████████████| 136 kB 45.1 MB/s 
     |████████████████████████████████| 636 kB 52.5 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
     |████████████████████████████████| 1.1 MB 30.6 MB/s 
     |████████████████████████████████| 96 kB 5.9 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 271 kB 66.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [2]:
#체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
#토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
#모델 입력값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [5]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

In [6]:
#그리드 서치
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )

## - - 이전 코드 끝 - -

앞의 코드에서 그리드 서치를 수행한 결과인 `generated_ids`는 토큰 인덱스 시퀀스여서 사람이 알아보기 어렵습니다.   
다음 코드를 실행하면 토크나이저가 `generated_ids`를 문장(문자열)으로 변환해 줍니다.

In [7]:
#토큰 인덱스를 문장으로 복원하기
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



여기서 "그럼, 그건 뭐예요?"는 모델이 생성한 문장입니다. 그리드 서치는 최고 확률을 내는 단어 시퀀스를 찾는 방법이므로 컨텍스트(`안녕하세요`)가 동일하다면 코드를 반복해서 실행해도 결과가 바뀌진 않습니다.

<**3단계**> **빔 서치하기**   

하지만 그리드 서치도 완벽한 대안은 아닙니다. 순간의 최선이 항상 전체의 최선이 될 수는 없기 때문입니다. 이번에 소개할 ***빔 서치(beam search)*** 는 빔 크기만큼의 선택지를 계산 범위에 넣습니다. 다음 그림은 빔 크기가 2인 빔 서치의 예시입니다.

<center><그림 - 빔 서치></center>

<p align="center"><img src="https://i.imgur.com/UCkkvzH.jpg">




모델은 `그` 다음에 올 단어로 `책`(0.5), `집`(0.4), `사람`(0.1) 순으로 예측했습니다. 우리는 빔 크기를 2로 설정해 두었으므로 예측 결과에서 확률이 높은 2개(`책, 집`)만 탐색 대상으로 남겨 두고, `사람`은 제거합니다. 그리고 모델에 `그 책`을 입력해 다음 토큰 등장 확률을 계산합니다.  
계산 결과는 다음과 같습니다.

|토큰 시퀀스|등장 확률|
|:---:|:---:|
|**그 책이**|**0.5 $\times$ 0.4 = 0.2**|
|그 책을|0.5 $\times$ 0.3 = 0.15|
|그 책 읽|0.5 $\times$ 0.3 = 0.15|


모델에 `그 집`을 입력해 다음 단어 시퀀스 확률을 계산합니다.  
계산 결과는 다음과 같습니다.

|토큰 시퀀스|등장 확률|
|:---:|:---:|
|**그 집에**|**0.4 $\times$ 0.7 = 0.28**|
|그 집은|0.4 $\times$ 0.2 = 0.08|
|그 집 사|0.4 $\times$ 0.1 = 0.04|

우리는 빔 크기를 2로 설정해 두었으므로 위 6가지 경우의 수에서 가장 확률이 높은 시퀀스 2개만을 남겨 둡니다. `그 집에`(0.28), `그 책이`(0.2)가 바로 그것입니다. 만일 빔 서치를 여기에서 그만둔다면 이 둘 가운데 확률값이 조금이라도 높은 `그 집에`가 최종 생성 결과가 됩니다.

빔 서치는 그리디 서치보다 계산량이 많은 편입니다. 그리디 서치가 매 순간 최고 확률을 내는 한 가지 경우의 수만 선택한다면, 빔 서치는 빔 크기만큼의 경우의 수를 고려하기 때문입니다. 하지만 빔 서치는 그리디 서치보다 조금이라도 더 높은 확률을 내는 문장을 생성할 수 있습니다. 앞에서 그리디 서치로 찾은 단어 시퀀스 확률은 0.2(`그 책이`)인 반면, 빔 서치는 이보다 약간 높은 0.28(`그 집에`)인 것을 확인할 수 있습니다.

진행 중인 실습에서 다음 코드를 실행하면 빔 서치를 수행합니다. 핵심 인자는 `do_sample=False`, `num_beams=3`입니다. `num_beams`는 빔 크기를 의미합니다. `num_beams=1`로 설정한다면 매 순간 최고 확률을 내는 단어만 선택한다는 뜻이므로 정확히 그리디 서치로 동작하게 됩니다.

In [8]:
#빔 서치
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        mix_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


<**4단계**> **반복되는 표현 줄이기**   

그리디 서치(`"그럼, 그건 뭐예요?"`)나 빔 서치(`"그렇지 않습니다."`) 모두 특정 표현이 반복되고 있음을 확인할 수 있습니다. 이럴 때 다음 코드를 실행하면 토큰이 n-gram 단위로 반복될 경우 모델이 계산한 결과를 무시하고 해당 n-gram의 마지막 토큰 등장 확률을 0으로 만들어 생성에서 배제하게 됩니다. 이 코드에서 핵심 인자는 `no_repeat_ngram_size=3`입니다.   
3개 이상의 토큰이 반복될 경우, 세 번째 토큰의 등장 확률을 0으로 만듭니다.  

cf) n-gram이란 n개의 연속적인 단어(또는 토큰)의 나열을 가리킵니다.

In [9]:
#반복 줄이기(1)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


결과는 위와 같은데요, 반복되는 n-gram 등장 확률을 인위적으로 조정할 뿐 최고 확률을 내는 토큰 시퀀스를 찾는다(`do_sample=False`)는 본질은 변함이 없으므로 코드를 반복해서 실행해도 생성 결과가 바뀌진 않습니다.